In [1]:
import re
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import os,sys
import tqdm
import pickle
import time
from concurrent.futures import ThreadPoolExecutor  
import concurrent.futures

In [2]:
rootPath ='D:/git/IoT_Sensors_Security_Analysis/data/perf/'

In [3]:
def from_trace_to_longstr(syscall_trace):
        tracestr = ''
        for syscall in syscall_trace:
            tracestr += syscall + ' '
        # print(tracestr)
        return tracestr

In [4]:
def read_all_rawdata(corpus_dataframe,corpus, rawdataPath, rawFileNames):    
    pool = ThreadPoolExecutor  (max_workers = 16)
    def read_file(inputFilePath):
        trace = pd.read_csv(inputFilePath)
        tr = trace['syscall'].tolist()             
        longstr = from_trace_to_longstr(tr)
        return (trace,longstr)
        # print(inputFilePath)
    def asyn_page(filenames):
        future_to_url  = dict()
        for i, url in enumerate(filenames):
            t = pool.submit(read_file, url)
            future_to_url[t] = url               
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                (trace,longstr) = data
                corpus_dataframe.append(trace)
                corpus.append(longstr)
            except Exception as exc:
                print('%r generated an exception: %s' % (filenames, exc))
    
    par = tqdm.tqdm(total = len(rawFileNames), ncols=100)
    # i = 0
    start, end = 0,0
    for n in range(0,len(rawFileNames),16):
        par.update(16)
        start = n
        if start + 16 < len(rawFileNames):
            end = start + 16
        else:
            end = len(rawFileNames)
        filenames = [rawdataPath + rawFileNames[i] for i in range(start, end)]
        asyn_page(filenames)
    par.close()
    pool.shutdown()
    print("Sub-process(es) done.")
    return corpus_dataframe, corpus

In [5]:
def create_onehot_encoding(total, index):
    onehot = []
    for i in range(0, total):
        if i == index:
            onehot.append(1)
        else:
            onehot.append(0)
    return onehot

In [6]:
def add_unk_to_dict(syscall_dict):
    total = len(syscall_dict)
    syscall_dict['unk'] = total
    syscall_dict_onehot = dict()
    for sc in syscall_dict:
        syscall_dict_onehot[sc] = create_onehot_encoding(total+1, syscall_dict[sc])
    return syscall_dict, syscall_dict_onehot

In [7]:
def replace_with_unk(syscall_trace, syscall_dict):
    for i, sc in enumerate(syscall_trace):
        if sc.lower() not in syscall_dict:
            syscall_trace[i] = 'unk'
    return syscall_trace

In [8]:
def trace_onehot_encoding(trace, syscall_dict_onehot):
    encoded_trace = []
    for syscall in trace:
        syscall = syscall.lower()
        if syscall.lower() in syscall_dict_onehot:
            one_hot = syscall_dict_onehot[syscall]
        else:
            syscall = 'UNK'
            one_hot = syscall_dict_onehot[syscall]
        encoded_trace.append(one_hot)
    return encoded_trace

In [9]:
def find_all_head(trace, head):
    starts, ends,se = [], [], []

    for i,s in enumerate(trace):
        if s == head:
            start=i
            starts.append(start)
            if len(starts) > 1:
                end = starts[-1] 
                ends.append(end)
        if i == len(trace)-1:
            end = len(trace)
            ends.append(end)
    se = [(starts[i], ends[i]) for i in range(0, len(starts))]
    return se

In [10]:
def get_distance( trace,head,tails):
    se = find_all_head(trace, head)
    mw = 12
    pool = ThreadPoolExecutor  (max_workers = mw)
    distances = []
    res = dict()
    def return_distance(tails, sort):
        distance = dict()
        for tail in tails:
            d = 0
            for j,t in enumerate(sort):
                if t==tail:
                    d += 1/(j)
            distance[tail] = d
        return distance

    def asyn_page(tails, sorts):
        future_to_url  = dict()
        for i, url in enumerate(sorts):
            t = pool.submit(return_distance, tails=tails, sort=url)
            future_to_url[t] = url               
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                distance = data
                distances.append(distance)
            except Exception as exc:
                print('generated an exception: %s' % (exc))
    start, end = 0,0
    for n in range(0, len(se), mw):
        start = n
        if start + mw < len(rawFileNames):
            end = start + mw
        else:
            end = len(rawFileNames)            
        sorts = [trace[s:e] for (s, e) in se[start:end]]
        asyn_page(tails, sorts)
    pool.shutdown()
    # print("Sub-process(es) done.")
    ds = pd.DataFrame(distances)
    for tail in tails:
        if tail in ds:
            res[(head, tail)] = sum(ds[tail])
        else:
            res[(head, tail)] = 0
    return res

In [11]:
def get_dependency_graph(trace,term_dict):
    dp = []
    dp_ = {}
    for head in term_dict:
        tails = list(term_dict.keys())
        tails.remove(head)  
        p = get_distance(trace,head,tails)
        dp_ = {**dp_, **p}
    
    for head in term_dict:
        d_ =  []
        for tail in term_dict:
            if head == tail:
                d_.append(0)
            else:
                d_.append(dp_[(head, tail)])
        dp.append(d_)
    return dp

In [12]:
def get_dict_sequence(trace,term_dict):
    dict_sequence = []
    for syscall in trace:
        if syscall in term_dict:
            dict_sequence.append(term_dict[syscall])
        else:
            dict_sequence.append(term_dict['unk'])
    return dict_sequence


In [13]:
def get_sequence_features(corpus_dataframe, syscall_dict, syscall_dict_onehot):
    one_hot_features = []
    dependency_graph_features = []
    dict_sequence_features = []
    pool = ThreadPoolExecutor  (max_workers = 16)

    def get_features(trace, syscall_dict, syscall_dict_onehot):
        syscall_one_hot = []
        dependency_graph = []
        dict_sequence = []
        syscall_trace = replace_with_unk(trace['syscall'].to_list(), syscall_dict)
        syscall_one_hot =  trace_onehot_encoding(syscall_trace, syscall_dict_onehot)
        dependency_graph = get_dependency_graph(syscall_trace,syscall_dict)
        dict_sequence=get_dict_sequence(syscall_trace,syscall_dict)
        return (syscall_one_hot, dependency_graph, dict_sequence)

    def asyn_page(cf):
        future_to_url  = dict()
        for i, url in enumerate(cf):
            t = pool.submit(get_features, url, syscall_dict, syscall_dict_onehot)
            future_to_url[t] = url               
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                (syscall_one_hot, dependency_graph, dict_sequence) = data
                one_hot_features.append(syscall_one_hot)
                dependency_graph_features.append(dependency_graph)
                dict_sequence_features.append(dict_sequence)
            except Exception as exc:
                print('generated an exception: %s' % ( exc))


    par = tqdm.tqdm(total = len(corpus_dataframe), ncols=100)
    # i = 0
    start, end = 0,0
    for n in range(0,len(corpus_dataframe),16):
        par.update(16)
        start = n
        if start + 16 < len(corpus_dataframe):
            end = start + 16
        else:
            end = len(corpus_dataframe)
        cf = corpus_dataframe[start: end]
        asyn_page(cf)
    par.close()   
    return one_hot_features, dependency_graph_features, dict_sequence_features

In [14]:
def run(testdevice, traindevice, tw):
    rawdataPath = rootPath+'{}/splited_1/{}/'.format(testdevice, tw)
    rawFileNames = os.listdir(rawdataPath)
    print(len(rawFileNames))

    rawdatas = dict()
    for n in range(0,len(rawFileNames),1000):
        start = n
        if start + 1000 < len(rawFileNames):
            end = start + 1000
        else:
            end = len(rawFileNames)
        rawdatas[n] = rawFileNames[start:end]

    corpus_dataframes, corpuses = {}, {}
    for rfs in rawdatas:
        corpus_dataframe, corpus = [],[]
        corpus_dataframe, corpus = read_all_rawdata(corpus_dataframe,corpus, rawdataPath, rawdatas[rfs])
        corpus_dataframes[rfs] = corpus_dataframe
        corpuses[rfs] = corpus
    
    corpus_dataframe, corpus = [],[]
    for rfs in corpus_dataframes:
        corpus_dataframe += corpus_dataframes[rfs]
        corpus += corpuses[rfs]
    
    # load the dict which is used for train model device
    dictPath = rootPath +'dicts/{}/{}/'.format(traindevice,tw)

    loc=open(dictPath+'syscall_dict.pk','rb')
    syscall_dict = pickle.load(loc)

    loc=open(dictPath+'syscall_dict_onehot.pk','rb')
    syscall_dict_onehot = pickle.load(loc)

    loc=open(dictPath+'ngrams_dict.pk','rb')
    ngrams_dict = pickle.load(loc)


    loc=open(dictPath+'countvectorizer.pk','rb')
    countvectorizer = pickle.load(loc)

    loc=open(dictPath+'hashingvectorizer.pk','rb')
    hashingvectorizer = pickle.load(loc)

    loc=open(dictPath+'tfidfvectorizer.pk','rb')
    tfidfvectorizer = pickle.load(loc)

    frequency_features = countvectorizer.transform(corpus)
    tfidf_features = tfidfvectorizer.transform(corpus)
    hashing_features = hashingvectorizer.transform(corpus)
    one_hot_features, dependency_graph_features, dict_sequence_features = get_sequence_features(corpus_dataframe, syscall_dict, syscall_dict_onehot)
    maltype = []
    ids = []
    for fi in rawFileNames:
        fis = fi.split('_')
        fn = fis[0]
        i = '{}_{}_{}'.format(fis[0], fis[2], fis[3])
        maltype.append(fn)
        ids.append(i)
    encoded_trace_df = pd.DataFrame([ids, maltype,frequency_features.toarray() ,tfidf_features.toarray(),hashing_features.toarray(), dependency_graph_features, one_hot_features, dict_sequence_features] ).transpose()
    encoded_trace_df.columns = ['ids', 'maltype',  'system calls frequency' ,'system calls tfidf','system calls hashing', 'system calls dependency graph', 'one hot encoding', 'dict index encoding']

    temppath  = rootPath + 'testset/train{}_test{}/'.format(traindevice, testdevice)
    encoded_trace_df.to_pickle(temppath+'test_{}_encoded_bow.pkl'.format(tw))

In [15]:
train_devices = ['pi3', 'pi4_2G']
test_devices = ['pi3', 'pi4_2G']
tws = [50, 60]

for traindevice in train_devices:
    for testdevice in test_devices:    
        for tw in tws:
            if testdevice == traindevice:
                continue
            elif traindevice=='pi3' and testdevice=='pi4_2G' and tw==50:
                continue
            elif traindevice=='pi3' and testdevice=='pi4_2G' and tw==60:
                continue
            else:
                print(testdevice, traindevice, tw)
                run(testdevice, traindevice, tw)

pi3 pi4_2G 50
3932


  0%|                                                                      | 0/1000 [00:00<?, ?it/s]C:\Python39\lib\concurrent\futures\thread.py:52: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  result = self.fn(*self.args, **self.kwargs)
1008it [01:41,  9.89it/s]


Sub-process(es) done.


1008it [01:47,  9.36it/s]


Sub-process(es) done.


...

generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined


100%|██████████████████████████████████████████████████████████▉| 3216/3217 [04:15<00:00, 13.15it/s]

generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined


3232it [04:16, 12.60it/s]

generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defined
generated an exception: name 'rawFileNames' is not defin

In [3]:
tw = 50
rawdataPath = rootPath+'data_1/'+str(tw)+'/'
# rawdataPath ='F:/temp/{}/'.format(tw)
rawFileNames = os.listdir(rawdataPath)
len(rawFileNames)

3932

In [5]:
rawdatas = dict()
for n in range(0,len(rawFileNames),1000):
    start = n
    if start + 1000 < len(rawFileNames):
        end = start + 1000
    else:
        end = len(rawFileNames)
    rawdatas[n] = rawFileNames[start:end]

In [6]:
corpus_dataframes, corpuses = {}, {}

In [8]:
for rfs in rawdatas:
    corpus_dataframe, corpus = [],[]
    corpus_dataframe, corpus = read_all_rawdata(corpus_dataframe,corpus, rawdataPath, rawdatas[rfs])
    corpus_dataframes[rfs] = corpus_dataframe
    corpuses[rfs] = corpus

  0%|                                                                      | 0/1000 [00:00<?, ?it/s]C:\Python39\lib\concurrent\futures\thread.py:52: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  result = self.fn(*self.args, **self.kwargs)
1008it [02:27,  6.83it/s]


Sub-process(es) done.


1008it [02:48,  5.97it/s]


Sub-process(es) done.


1008it [03:18,  5.08it/s]


Sub-process(es) done.


944it [04:11,  3.76it/s]

Sub-process(es) done.


In [9]:
corpus_dataframe, corpus = [],[]
for rfs in corpus_dataframes:
    corpus_dataframe += corpus_dataframes[rfs]
    corpus += corpuses[rfs]

In [10]:
dictPath = rootPath +'dicts/pi4_2G/'+str(tw)+'/'

loc=open(dictPath+'syscall_dict.pk','rb')
syscall_dict = pickle.load(loc)

loc=open(dictPath+'syscall_dict_onehot.pk','rb')
syscall_dict_onehot = pickle.load(loc)

loc=open(dictPath+'ngrams_dict.pk','rb')
ngrams_dict = pickle.load(loc)


loc=open(dictPath+'countvectorizer.pk','rb')
countvectorizer = pickle.load(loc)

loc=open(dictPath+'hashingvectorizer.pk','rb')
hashingvectorizer = pickle.load(loc)

loc=open(dictPath+'tfidfvectorizer.pk','rb')
tfidfvectorizer = pickle.load(loc)

In [11]:
frequency_features = countvectorizer.transform(corpus)
tfidf_features = tfidfvectorizer.transform(corpus)
hashing_features = hashingvectorizer.transform(corpus)

In [21]:
one_hot_features, dependency_graph_features, dict_sequence_features = get_sequence_features(corpus_dataframe, syscall_dict, syscall_dict_onehot)

3936it [1:46:53,  1.63s/it]


In [22]:
maltype = []
ids = []
for fi in rawFileNames:
    fis = fi.split('_')
    fn = fis[0]
    i = '{}_{}_{}'.format(fis[0], fis[2], fis[3])
    maltype.append(fn)
    ids.append(i)

In [23]:
encoded_trace_df = pd.DataFrame([ids, maltype,frequency_features.toarray() ,tfidf_features.toarray(),hashing_features.toarray(), dependency_graph_features, one_hot_features, dict_sequence_features] ).transpose()
encoded_trace_df.columns = ['ids', 'maltype',  'system calls frequency' ,'system calls tfidf','system calls hashing', 'system calls dependency graph', 'one hot encoding', 'dict index encoding']

In [25]:
temppath  = rootPath + 'testset/trainpi4_2G_testpi3/'
encoded_trace_df.to_pickle(temppath+'test_{}_encoded_bow.pkl'.format(tw))